In [1]:
!git clone https://github.com/guijiejie/DCMD-main.git

Cloning into 'DCMD-main'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 74 (delta 22), reused 48 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 262.60 KiB | 4.61 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
!ls /kaggle/working/

# **Pip**

In [2]:
!pip install appdirs==1.4.4 docker-pycreds==0.4.0 gitdb==4.0.10 gitpython==3.1.32 joblib==1.3.1 numpy==1.25.2 pathtools==0.1.2 protobuf==4.23.4 scikit-learn==1.3.0 scipy==1.11.1 sentry-sdk==1.29.2 setproctitle==1.3.2 smmap==5.0.0 threadpoolctl==3.2.0 wandb==0.15.8
!pip install --upgrade pytorch-lightning pyyaml wandb pandas numpy matplotlib matplotlib-inline scikit-learn tqdm
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 96.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 51.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8793 sha256=ab5f9f85d4cb90d9bc5d01e1c1866071e546e3c980

# **Replace code to run properly**

In [8]:
%%writefile /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml 
### Experiment configuration

## General settings
split: 'train' # data split; choices ['train', 'test']
debug: false # if true, load only a few data samples
seed: 999
validation: false # use validation; only for UBnormal
use_hr: false # for validation and test on UBnormal

## Computational resources
accelerator: 'gpu'
devices: [0] # indices of cuda devices to use

## Paths
dir_name: 'train_experiment' # name of the directory of the current experiment
data_dir: '/kaggle/input/avenue/Avenue' # path to the data
exp_dir: '/kaggle/working/DCMD-main/checkpoints' # path to the directory that will contain the current experiment directory
test_path: '/kaggle/input/avenue/Avenue/testing/test_frame_mask' # path to the test data
load_ckpt: 'best.ckpt' # name of the checkpoint to load at inference time
create_experiment_dir: true

## WANDB configuration
use_wandb: false
project_name: "project_name"
wandb_entity: "entity_name"
group_name: "group_name"
use_ema: false

##############################


### Model's configuration

## U-Net's configuration
dropout: 0. # probability of dropout
conditioning_strategy: 'inject'
## Rec configuration
h_dim: 512 # dimension of the bottleneck at the end of the encoder of the conditioning network
latent_dim: 256 # dimension of the latent space of the conditioning encoder
channels: [512,256,512] # channels for the encoder

##############################


### Training's configuration

## Diffusion's configuration
noise_steps: 10 # how many diffusion steps to perform

### Optimizer and scheduler's configuration
n_epochs: 10
opt_lr: 0.001

## Losses' configuration
loss_fn: 'smooth_l1' # loss function; choices ['mse', 'l1', 'smooth_l1']

##############################


### Inference's configuration
n_generated_samples: 50 # number of samples to generate
model_return_value: 'loss' # choices ['loss', 'poses', 'all']; if 'loss', the model will return the loss;
                           # if 'poses', the model will return the generated poses; 
                           # if 'all', the model will return both the loss and the generated poses
aggregation_strategy: 'best' # choices ['best', 'mean', 'median', 'random']; if 'best', the best sample will be selected; 
                             # if 'mean', the mean of loss of the samples will be selected; 
                             # if 'median', the median of the loss of the samples will be selected; 
                             # if 'random', a random sample will be selected;
                             # if 'mean_poses', the mean of the generated poses will be selected;
                             # if 'median_poses', the median of the generated poses will be selected;
                             # if 'all', all the generated poses will be selected
filter_kernel_size: 30 # size of the kernel to use for smoothing the anomaly score of each clip
frames_shift: 6 # it compensates the shift of the anomaly score due to the sliding window; 
                # in conjuction with pad_size and filter_kernel_size, it strongly depends on the dataset
save_tensors: false # if true, save the generated tensors for faster inference
load_tensors: false # if true, load the generated tensors for faster inference

##############################


### Dataset's configuration

## Important parameters
dataset_choice: 'HR-Avenue'
seg_len: 7 # length of the window (his+pre)
vid_res: [640,360]
batch_size: 2048
pad_size: 12 # size of the padding 

## Other parameters
headless: false # remove the keypoints of the head
hip_center: false # center the keypoints on the hip
kp18_format: false # use the 18 keypoints format
normalization_strategy: 'robust' # use 'none' to avoid normalization, 'robust' otherwise
num_coords: 2 # number of coordinates to use
num_transform: 5 # number of transformations to apply
num_workers: 4
seg_stride: 1
seg_th: 0
start_offset: 0
symm_range: true
use_fitted_scaler: false

## New configuration
n_his: 3
padding: 'LastFrame'
## translinear configuration
num_layers: 6
num_heads: 8
latent_dims: 512
loss_1_series_weight: 0.01
loss_1_prior_weight: 0
loss_2_series_weight: 0
loss_2_prior_weight: 0.01

Overwriting /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml


In [9]:
%%writefile /kaggle/working/DCMD-main/models/transformer.py
import torch
import torch.nn.functional as F
from torch import layer_norm, nn
import numpy as np
from typing import List, Tuple, Union

import math


def timestep_embedding(timesteps, dim, max_period=10000):
    """
    Create sinusoidal timestep embeddings.
    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
    half = dim // 2
    freqs = torch.exp(
        -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
    ).to(device=timesteps.device)
    args = timesteps[:, None].float() * freqs[None]
    embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    if dim % 2:
        embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
    return embedding


def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad for all the networks.

    Args:
        nets (nn.Module | list[nn.Module]): A list of networks or a single
            network.
        requires_grad (bool): Whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad


def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module


class StylizationBlock(nn.Module):

    def __init__(self, latent_dim, time_embed_dim, dropout):
        super().__init__()
        self.emb_layers = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_embed_dim, 2 * latent_dim),
        )
        self.norm = nn.LayerNorm(latent_dim)
        self.out_layers = nn.Sequential(
            nn.SiLU(),
            nn.Dropout(p=dropout),
            zero_module(nn.Linear(latent_dim, latent_dim)),
        )

    def forward(self, h, emb):
        """
        h: B, T, D
        emb: B, D
        """
        # B, 1, 2D
        emb_out = self.emb_layers(emb).unsqueeze(1)
        # scale: B, 1, D / shift: B, 1, D
        scale, shift = torch.chunk(emb_out, 2, dim=2)
        # B, T, D
        h = self.norm(h) * (1 + scale) + shift
        h = self.out_layers(h)
        return h


class FFN(nn.Module):

    def __init__(self, latent_dim, ffn_dim, dropout, time_embed_dim):
        super().__init__()
        self.linear1 = nn.Linear(latent_dim, ffn_dim)
        self.linear2 = zero_module(nn.Linear(ffn_dim, latent_dim))
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        self.proj_out = StylizationBlock(latent_dim, time_embed_dim, dropout)

    def forward(self, x, emb):
        """
            x: B, T, D (D=latent_dim)
        """
        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        y = x + self.proj_out(y, emb)
        return y


class TemporalSelfAttention(nn.Module):

    def __init__(self, n_frames, latent_dim, num_head, dropout, time_embed_dim, output_attention = True):
        super().__init__()
        self.num_head = num_head
        self.output_attention = output_attention
        self.norm = nn.LayerNorm(latent_dim)
        self.query = nn.Linear(latent_dim, latent_dim, bias=False)
        self.key = nn.Linear(latent_dim, latent_dim, bias=False)
        self.value = nn.Linear(latent_dim, latent_dim, bias=False)
        self.sigma_projection = nn.Linear(latent_dim, num_head, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.proj_out = StylizationBlock(latent_dim, time_embed_dim, dropout)
        n_frames = n_frames
        self.distances = torch.zeros((n_frames, n_frames)).cuda(0)

        for i in range(n_frames):
            for j in range(n_frames):
                self.distances[i][j] = abs(i - j)

    def forward(self, x, emb):
        """
        x: B, T, D (D=latent_dim)
        """
        B, T, D = x.shape
        H = self.num_head

        ## series-association
        # B, T, 1, D
        query = self.query(self.norm(x)).unsqueeze(2)
        # B, 1, T, D
        key = self.key(self.norm(x)).unsqueeze(1)
        # B, T, H, D/H
        query = query.view(B, T, H, -1)
        key = key.view(B, T, H, -1)
        scale = 1. / math.sqrt(D/H)
        # B, H, T, T
        scores = torch.einsum('bnhd,bmhd->bhnm', query, key) / math.sqrt(D // H)
        attention = scale * scores
        # B, H, T, T
        series = self.dropout(F.softmax(attention, dim=-1))

        ## prior-association
        sigma = self.sigma_projection(x).view(B, T, H)  # B, T, H
        sigma = sigma.transpose(1, 2)  # B T H ->  B H T
        sigma = torch.sigmoid(sigma * 5) + 1e-5
        sigma = torch.pow(3, sigma) - 1
        sigma = sigma.unsqueeze(-1).repeat(1, 1, 1, T)  # B, H, T, T
        prior = self.distances.unsqueeze(0).unsqueeze(0).repeat(sigma.shape[0], sigma.shape[1], 1, 1) # B, H, T, T
        prior = 1.0 / (math.sqrt(2 * math.pi) * sigma) * torch.exp(-prior ** 2 / 2 / (sigma ** 2)).cuda(0) # B, H, T, T

        # B, T, H, D/H
        value = self.value(self.norm(x)).view(B, T, H, -1)
        # B, T, D
        y = torch.einsum('bhnm,bmhd->bnhd', series, value).reshape(B, T, D)
        y = x + self.proj_out(y, emb)

        if self.output_attention:
            return y.contiguous(), series, prior, sigma
        else:
            return y.contiguous(), None

class TemporalDiffusionTransformerDecoderLayer(nn.Module):

    def __init__(self,
                 n_frames = 7,
                 latent_dim=16,
                 time_embed_dim=16,
                 ffn_dim=32,
                 num_head=4,
                 dropout=0.5
                 ):
        super().__init__()
        self.sa_block = TemporalSelfAttention(
            n_frames, latent_dim, num_head, dropout, time_embed_dim)
        self.ffn = FFN(latent_dim, ffn_dim, dropout, time_embed_dim)

    def forward(self, x, emb):
        x, series, prior, sigma = self.sa_block(x, emb)
        x = self.ffn(x, emb)
        return x, series, prior, sigma


class MotionTransformer(nn.Module):
    def __init__(self,
                 input_feats,
                 num_frames=7,
                 latent_dim=16,
                 ff_size=32,
                 num_layers=8,
                 num_heads=8,
                 dropout=0.2,
                 activation="gelu",
                 output_attention = True,
                 device: Union[str, torch.DeviceObjType] = 'cpu',
                 inject_condition: bool = False,
                 **kargs):
        super().__init__()


        self.input_feats = input_feats # 34
        self.num_frames = num_frames
        self.latent_dim = latent_dim
        self.ff_size = ff_size
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dropout = dropout
        self.activation = activation
        self.output_attention = output_attention
        self.device = device
        self.time_embed_dim = latent_dim
        self.inject_condition = inject_condition

        self.build_model()

    def build_model(self):
        self.sequence_embedding = nn.Parameter(torch.randn(self.num_frames, self.latent_dim))

        # Input Embedding
        self.joint_embed = nn.Linear(self.input_feats, self.latent_dim)
        self.cond_embed = nn.Linear(256, self.time_embed_dim)

        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, self.time_embed_dim),
            nn.SiLU(),
            nn.Linear(self.time_embed_dim, self.time_embed_dim),
        )

        self.temporal_decoder_blocks = nn.ModuleList()
        for i in range(self.num_layers):
            self.temporal_decoder_blocks.append(
                TemporalDiffusionTransformerDecoderLayer(
                    n_frames=self.num_frames,
                    latent_dim=self.latent_dim,
                    time_embed_dim=self.time_embed_dim,
                    ffn_dim=self.ff_size,
                    num_head=self.num_heads,
                    dropout=self.dropout,
                )
            )
        # Output Module
        self.out = zero_module(nn.Linear(self.latent_dim, self.input_feats))


    def forward(self, x, timesteps, condition_data:torch.Tensor=None):
        """
        x: B, T, D (D=C*V)
        """
        B, T = x.shape[0], x.shape[1]

        # B, latent_dim
        emb = self.time_embed(timestep_embedding(timesteps, self.latent_dim))

        # Add conditioning signal
        if self.inject_condition:
            condition_data = self.cond_embed(condition_data)
            emb = emb + condition_data

        # B, T, latent_dim
        h = self.joint_embed(x)
        h = h + self.sequence_embedding.unsqueeze(0)[:, :T, :]
        
        i = 0
        prelist = []
        series_list = []
        prior_list = []
        sigma_list = []
        for module in self.temporal_decoder_blocks:
            if i < (self.num_layers // 2):
                prelist.append(h)
                h, series, prior, sigmas = module(h, emb) # B, T, latent_dim
                series_list.append(series)
                prior_list.append(prior)
                sigma_list.append(sigmas)
            elif i >= (self.num_layers // 2):
                h, series, prior, sigmas = module(h, emb)
                h += prelist[-1]
                series_list.append(series)
                prior_list.append(prior)
                sigma_list.append(sigmas)
                prelist.pop()
            i += 1

        # B, T, C*V
        output = self.out(h).view(B, T, -1).contiguous()
        if self.output_attention:
            return output, series_list, prior_list, sigma_list
        return output

Overwriting /kaggle/working/DCMD-main/models/transformer.py


In [10]:
%%writefile /kaggle/working/DCMD-main/train_DCMD.py
import argparse
import os
import random

import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from models.dcmd import DCMD
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.strategies import DDPStrategy
from utils.argparser import init_args
from utils.dataset import get_dataset_and_loader
from utils.ema import EMACallback


if __name__== '__main__':

    # Parse command line arguments and load config file
    parser = argparse.ArgumentParser(description='Pose_AD_Experiment')
    parser.add_argument('-c', '--config', type=str, required=True,
                        default='/your_default_config_file_path')
    
    args = parser.parse_args()
    config_path = args.config
    args = yaml.load(open(args.config), Loader=yaml.FullLoader)
    args = argparse.Namespace(**args)
    args = init_args(args) 
    # Save config file to ckpt_dir
    os.system(f'cp {config_path} {os.path.join(args.ckpt_dir, "config.yaml")}')     
    
    # Set seeds    
    torch.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed) 
    pl.seed_everything(args.seed)

    # Set callbacks and logger
    if (hasattr(args, 'diffusion_on_latent') and args.stage == 'pretrain'):
        monitored_metric = 'pretrain_rec_loss'
        metric_mode = 'min'
    elif args.validation:
        monitored_metric = 'AUC'
        metric_mode = 'max'
    else:
        monitored_metric = 'loss'
        metric_mode = 'min'
    callbacks = [ModelCheckpoint(
                    dirpath=args.ckpt_dir,
                    save_top_k=2,
                    save_last=True,
                    monitor=monitored_metric,
                    mode=metric_mode
                )]

    
    callbacks += [EMACallback()] if args.use_ema else [] # Use to achieve exponential moving average
    
    if args.use_wandb:
        callbacks += [LearningRateMonitor(logging_interval='step')]
        wandb_logger = WandbLogger(project=args.project_name, group=args.group_name, entity=args.wandb_entity, 
                                   name=args.dir_name, config=vars(args), log_model='all')
    else:
        wandb_logger = False

    # Get dataset and loaders
    _, train_loader, _, val_loader = get_dataset_and_loader(args, split=args.split, validation=args.validation)
    
    # Initialize model and trainer
    model = DCMD(args)
    
    trainer = pl.Trainer(accelerator=args.accelerator, devices=args.devices, default_root_dir=args.ckpt_dir, max_epochs=args.n_epochs, 
                         logger=wandb_logger, callbacks=callbacks, strategy=DDPStrategy(find_unused_parameters=False),
                         log_every_n_steps=20, num_sanity_val_steps=0, deterministic=True)
    
    # Train the model    
    trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Overwriting /kaggle/working/DCMD-main/train_DCMD.py


In [11]:
%%writefile /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml
### Experiment configuration

## General settings
split: 'test' # data split; choices ['train', 'test']
debug: false # if true, load only a few data samples
seed: 999
validation: false # use validation; only for UBnormal
use_hr: false # for validation and test on UBnormal

## Computational resources
accelerator: 'gpu'
devices: [0] # indices of cuda devices to use

## Paths
dir_name: 'test_experiment' # name of the directory of the current experiment
data_dir: '/kaggle/input/avenue/Avenue' # path to the data
exp_dir: '/kaggle/working/DCMD-main/checkpoints' # path to the directory that will contain the current experiment directory
test_path: '/kaggle/input/avenue/Avenue/testing/test_frame_mask' # path to the test data
load_ckpt: 'last.ckpt' # name of the checkpoint to load at inference time
create_experiment_dir: false # if true, create a new directory for the current experiment

## WANDB configuration
use_wandb: false
project_name: "project_name"
wandb_entity: "entity_name"
group_name: "group_name"
use_ema: false

##############################


### Model's configuration

## U-Net's configuration
dropout: 0. # probability of dropout
conditioning_strategy: 'inject'
## Rec configuration
h_dim: 512 # dimension of the bottleneck at the end of the encoder of the conditioning network
latent_dim: 256 # dimension of the latent space of the conditioning encoder
channels: [512,256,512] # channels for the encoder

##############################


### Training's configuration

## Diffusion's configuration
noise_steps: 10 # how many diffusion steps to perform

### Optimizer and scheduler's configuration
n_epochs: 10
opt_lr: 0.001

## Losses' configuration
loss_fn: 'smooth_l1' # loss function; choices ['mse', 'l1', 'smooth_l1']

##############################


### Inference's configuration
n_generated_samples: 50 # number of samples to generate
model_return_value: 'loss' # choices ['loss', 'poses', 'all']; if 'loss', the model will return the loss;
                           # if 'poses', the model will return the generated poses; 
                           # if 'all', the model will return both the loss and the generated poses
aggregation_strategy: 'best' # choices ['best', 'mean', 'median', 'random']; if 'best', the best sample will be selected; 
                             # if 'mean', the mean of loss of the samples will be selected; 
                             # if 'median', the median of the loss of the samples will be selected; 
                             # if 'random', a random sample will be selected;
                             # if 'mean_poses', the mean of the generated poses will be selected;
                             # if 'median_poses', the median of the generated poses will be selected;
                             # if 'all', all the generated poses will be selected
filter_kernel_size: 30 # size of the kernel to use for smoothing the anomaly score of each clip
frames_shift: 6 # it compensates the shift of the anomaly score due to the sliding window; 
                # in conjuction with pad_size and filter_kernel_size, it strongly depends on the dataset
save_tensors: true # if true, save the generated tensors for faster inference
load_tensors: false # if true, load the generated tensors for faster inference

##############################


### Dataset's configuration

## Important parameters
dataset_choice: 'HR-Avenue'
seg_len: 7 # length of the window (cond+noised)
vid_res: [640,360]
batch_size: 2048
pad_size: 12 # size of the padding

## Other parameters
headless: false # remove the keypoints of the head
hip_center: false # center the keypoints on the hip
kp18_format: false # use the 18 keypoints format
normalization_strategy: 'robust' # use 'none' to avoid normalization, 'robust' otherwise
num_coords: 2 # number of coordinates to use
num_transform: 5 # number of transformations to apply
num_workers: 4
seg_stride: 1
seg_th: 0
start_offset: 0
symm_range: true
use_fitted_scaler: false

## New configuration
n_his: 3
padding: 'LastFrame'
## translinear configuration
num_layers: 6
num_heads: 8
latent_dims: 512
loss_1_series_weight: 0.01
loss_1_prior_weight: 0
loss_2_series_weight: 0
loss_2_prior_weight: 0.01

Overwriting /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml


In [12]:
%%writefile /kaggle/working/DCMD-main/models/dcmd.py
import argparse
import os
from math import prod
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt

from models.stsae.stsae import STSAE
from sklearn.metrics import roc_curve, roc_auc_score
from torch.optim import Adam
from tqdm import tqdm

from utils.diffusion_utils import Diffusion
from utils.eval_utils import (compute_var_matrix, filter_vectors_by_cond,
                              get_avenue_mask, get_hr_ubnormal_mask, pad_scores, score_process)
from utils.model_utils import processing_data, my_kl_loss
from models.transformer import MotionTransformer
from utils.tools import get_dct_matrix, generate_pad, padding_traj


class DCMD(pl.LightningModule):

    losses = {'l1': nn.L1Loss, 'smooth_l1': nn.SmoothL1Loss, 'mse': nn.MSELoss}
    conditioning_strategies = {'inject': 'inject'}

    def __init__(self, args: argparse.Namespace) -> None:
        """
        This class implements DCMD model.

        Args:
            args (argparse.Namespace): arguments containing the hyperparameters of the model
        """

        super(DCMD, self).__init__()

        ## Log the hyperparameters of the model
        self.save_hyperparameters(args)

        ## Set the internal variables of the model
        # Data parameters
        self.n_frames = args.seg_len
        self.num_coords = args.num_coords
        self.n_joints = self._infer_number_of_joint(args)

        ## Model parameters
        # Main network
        self.dropout = args.dropout
        self.conditioning_strategy = self.conditioning_strategies[args.conditioning_strategy]
        # Conditioning network
        self.cond_h_dim = args.h_dim
        self.cond_latent_dim = args.latent_dim
        self.cond_channels = args.channels
        self.cond_dropout = args.dropout

        ## Training and inference parameters
        self.learning_rate = args.opt_lr
        self.loss_fn = self.losses[args.loss_fn](reduction='none')
        self.noise_steps = args.noise_steps
        self.aggregation_strategy = args.aggregation_strategy
        self.n_generated_samples = args.n_generated_samples
        self.model_return_value = args.model_return_value
        self.gt_path = args.gt_path
        self.split = args.split
        self.use_hr = args.use_hr
        self.ckpt_dir = args.ckpt_dir
        self.save_tensors = args.save_tensors
        self.num_transforms = args.num_transform
        self.anomaly_score_pad_size = args.pad_size
        self.anomaly_score_filter_kernel_size = args.filter_kernel_size
        self.anomaly_score_frames_shift = args.frames_shift
        self.dataset_name = args.dataset_choice

        # New parameters
        self.n_his = args.n_his
        self.padding = args.padding
        self.num_layers = args.num_layers
        self.num_heads = args.num_heads
        self.latent_dims = args.latent_dims
        self.automatic_optimization = False
        self.loss_1_series_weight = args.loss_1_series_weight
        self.loss_1_prior_weight = args.loss_1_prior_weight
        self.loss_2_series_weight = args.loss_2_series_weight
        self.loss_2_prior_weight = args.loss_2_prior_weight
        self.idx_pad, self.zero_index = generate_pad(self.padding, self.n_his, self.n_frames-self.n_his)

        ## Set the noise scheduler for the diffusion process
        self._set_diffusion_variables()

        ## Build the model
        self.build_model()


    def build_model(self) -> None:
        """
        Build the model according to the specified hyperparameters.
        """

        # Prediction Model
        pre_model = MotionTransformer(
            input_feats=2 * self.n_joints,
            num_frames=self.n_frames,
            num_layers=self.num_layers,
            num_heads=self.num_heads,
            latent_dim=self.latent_dims,
            dropout=self.dropout,
            device=self.device,
            inject_condition=(self.conditioning_strategy == 'inject')
        )

        # Reconstruction Model
        rec_model = STSAE(
            c_in=self.num_coords,
            h_dim=self.cond_h_dim,
            latent_dim=self.cond_latent_dim,
            n_frames=self.n_his,
            dropout=self.cond_dropout,
            n_joints=self.n_joints,
            layer_channels=self.cond_channels,
            device=self.device)

        self.pre_model, self.rec_model = pre_model, rec_model


    def forward(self, input_data: List[torch.Tensor], aggr_strategy: str = None, return_: str = None) -> List[torch.Tensor]:
        """
        Forward pass of the model.
        """

        ## Unpack data: tensor_data is the input data, meta_out is a list of metadata
        tensor_data, meta_out = self._unpack_data(input_data)
        B = tensor_data.shape[0]

        ## Select frames to reconstruct and to predict
        history_data = tensor_data[:, :, :self.n_his, :]
        x_0 = padding_traj(history_data, self.padding, self.idx_pad, self.zero_index)

        generated_xs = []
        # Generate m future predictions
        for _ in range(self.n_generated_samples):

            ## Reconstruction —— AE model
            condition_embedding, rec_his_data = self.rec_model(history_data)

            ## Prediction —— diffusion model
            ## DCT transformation
            dct_m, idct_m = get_dct_matrix(self.n_frames)
            dct_m_all = dct_m.float().to(self.device)
            idct_m_all = idct_m.float().to(self.device)
            # (B, C, T, V) -> (B, T, V, C)
            x = x_0.permute(0, 2, 3, 1).contiguous()
            # (B, T, V, C) -> (B, T, C*V)
            x = x.reshape([x.shape[0], self.n_frames, -1])
            y = torch.matmul(dct_m_all, x)  # [B, T, C*V]

            ## Generate gaussian noise of the same shape as the y
            y_d = torch.randn_like(y, device=self.device)

            ## (t ∈ T, T-1, ..., 1)
            for i in reversed(range(1, self.noise_steps)):

                ### Prediction (Two branches)
                ## Set the time step
                t = torch.full(size=(B,), fill_value=i, dtype=torch.long, device=self.device)
                t_prev = torch.full(size=(B,), fill_value=i, dtype=torch.long, device=self.device)
                t_prev[0] = 0

                ## Generate gaussian noise of the same shape as the predicted noise
                noise_pre = torch.randn_like(y_d, device=self.device) if i > 1 else torch.zeros_like(y_d, device=self.device)

                ## First branch
                # Predict the noise
                predicted_noise_pre, series, prior, _ = self.pre_model(y_d, t, condition_data=condition_embedding)
                # Get the alpha and beta values and expand them to the shape of the predicted noise
                alpha_pre = self._alpha[t][:, None, None]
                alpha_hat_pre = self._alpha_hat[t][:, None, None]
                beta_pre = self._beta[t][:, None, None]
                # Recover the predicted sequence
                y_d = (1 / torch.sqrt(alpha_pre)) * (y_d - ((1 - alpha_pre) / (torch.sqrt(1 - alpha_hat_pre))) * predicted_noise_pre) \
                    + torch.sqrt(beta_pre) * noise_pre
                ## Second branch
                alpha_hat_prev = self._alpha_hat[t_prev][:, None, None]
                # Add noise
                y_n = (torch.sqrt(alpha_hat_prev) * y) + (torch.sqrt(1 - alpha_hat_prev) * noise_pre)
                ## Mask completion
                # Get M values
                mask = torch.zeros_like(x, device=self.device) # [batch, T, C*V]
                for m in range(0, self.n_his):
                    mask[:, m, :] = 1
                # iDCT transformation
                y_d_idct = torch.matmul(idct_m_all, y_d)
                y_n_idct = torch.matmul(idct_m_all, y_n)
                # mask-mul
                m_mul_y_n = torch.mul(mask, y_n_idct)
                m_mul_y_d = torch.mul((1-mask), y_d_idct)
                # together
                m_y = m_mul_y_d + m_mul_y_n
                # DCT again
                y_d = torch.matmul(dct_m_all, m_y)

            # iDCT
            pre_future_data = torch.matmul(idct_m_all, y_d)
            # (B, T, C*V) -> (B, T, V, C)
            pre_future_data = pre_future_data.reshape(pre_future_data.shape[0], pre_future_data.shape[1], -1, 2)
            # (B, T, V, C) -> (B, C, T, V)
            pre_future_data = pre_future_data.permute(0, 3, 1, 2).contiguous()
            # select future sequences
            pre_future_data = pre_future_data[:,:,self.n_his:,:]

            ## Reconstruction + Prediction
            xs = torch.cat((rec_his_data, pre_future_data), dim=2)

            generated_xs.append(xs)

        selected_x, loss_of_selected_x = self._aggregation_strategy(generated_xs, tensor_data, aggr_strategy)

        return self._pack_out_data(selected_x, loss_of_selected_x, [tensor_data] + meta_out, return_=return_)


    def training_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.float32:
        """
        Training step of the model.
        """

        ## Get the optimizer returned in configuration_optimizers()
        opt = self.optimizers()

        ## Unpack data: tensor_data is the input data
        tensor_data, _ = self._unpack_data(batch)

        ## Select frames to reconstruct and to predict
        history_data = tensor_data[:, :, :self.n_his, :] # Used for rec (first n_his)
        x_0 = tensor_data # Used for pre（all）

        ## Reconstruction
        # Encode the history data
        condition_embedding, rec_his_data = self.rec_model(history_data)
        # Compute the rec_loss
        rec_loss = torch.mean(self.loss_fn(rec_his_data, history_data))
        self.log('rec_loss', rec_loss)

        ## Prediction
        # DCT transformation
        dct_m, _ = get_dct_matrix(self.n_frames)
        dct_m_all = dct_m.float().to(self.device)
        # (B, C, T, V) -> (B, T, V, C)
        x = x_0.permute(0, 2, 3, 1).contiguous()
        # (B, T, V, C) -> (B, T, C*V)
        x = x.reshape([x.shape[0], self.n_frames, -1]) # [batch, T, C*V]
        y_0 = torch.matmul(dct_m_all, x)

        # Sample the time steps and corrupt the data
        t = self.noise_scheduler.sample_timesteps(y_0.shape[0]).to(self.device)
        y_t, pre_noise = self.noise_scheduler.noise_motion(y_0, t) # (B, T, C*(V-1))

        # Predict the noise
        pre_predicted_noise, series, prior, _ = self.pre_model(y_t, t, condition_data=condition_embedding)

        # Compute the pre_loss
        # Calculate Association discrepancy
        series_loss = 0.0
        prior_loss = 0.0
        for u in range(len(prior)):
            # Pdetach, S <-> Maximize
            series_loss += \
                (torch.mean(my_kl_loss(
                    series[u],
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)).detach()))
                + torch.mean(my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)).detach(),
                    series[u])))
            # P, Sdetach <-> Minimize
            prior_loss += \
                (torch.mean(my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)),
                    series[u].detach()))
                + torch.mean(my_kl_loss(
                    series[u].detach(),
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)))))
        series_loss = series_loss / len(prior)
        prior_loss = prior_loss / len(prior)

        pre_loss = torch.mean(self.loss_fn(pre_predicted_noise, pre_noise))
        self.log('pre_loss', pre_loss)

        ## Compute loss1 & loss2
        loss1 = rec_loss + pre_loss \
                - self.loss_1_series_weight * series_loss \
                + self.loss_1_prior_weight * prior_loss
        self.log('loss1', loss1)
        loss2 = rec_loss + pre_loss \
                + self.loss_2_prior_weight * prior_loss \
                + self.loss_2_series_weight * series_loss
        self.log('loss2', loss2)

        ## Minimax strategy
        self.manual_backward(loss1, retain_graph=True)
        self.manual_backward(loss2)
        opt.step()
        opt.zero_grad()


    def test_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        """
        Test step of the model. It saves the output of the model and the input data as
        List[torch.Tensor]: [predicted poses and the loss, tensor_data, transformation_idx, metadata, actual_frames]

        Args:
            batch (List[torch.Tensor]): list containing the following tensors:
                                         - tensor_data: tensor of shape (B, C, T, V) containing the input sequences
                                         - transformation_idx
                                         - metadata
                                         - actual_frames
            batch_idx (int): index of the batch
        """

        self._test_output_list.append(self.forward(batch))
        return


    def on_test_epoch_start(self) -> None:
        """
        Called when the test epoch begins.
        """

        super().on_test_epoch_start()
        self._test_output_list = []
        return


    def on_test_epoch_end(self) -> float:
        """
        Test epoch end of the model.

        Returns:
            float: test auc score
        """

        out, gt_data, trans, meta, frames = processing_data(self._test_output_list)
        del self._test_output_list
        if self.save_tensors:
            tensors = {'prediction': out, 'gt_data': gt_data,
                       'trans': trans, 'metadata': meta, 'frames': frames}
            self._save_tensors(tensors, split_name=self.split, aggr_strategy=self.aggregation_strategy, n_gen=self.n_generated_samples)
        auc_score = self.post_processing(out, gt_data, trans, meta, frames)
        self.log('AUC', auc_score)
        return auc_score


    def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        """
        Validation step of the model. It saves the output of the model and the input data as
        List[torch.Tensor]: [predicted poses and the loss, tensor_data, transformation_idx, metadata, actual_frames]

        Args:
            batch (List[torch.Tensor]): list containing the following tensors:
                                         - tensor_data: tensor of shape (B, C, T, V) containing the input sequences
                                         - transformation_idx
                                         - metadata
                                         - actual_frames
            batch_idx (int): index of the batch
        """

        self._validation_output_list.append(self.forward(batch))
        return


    def on_validation_epoch_start(self) -> None:
        """
        Called when the test epoch begins.
        """

        super().on_validation_epoch_start()
        self._validation_output_list = []
        return


    def on_validation_epoch_end(self) -> float:
        """
        Validation epoch end of the model.

        Returns:
            float: validation auc score
        """

        out, gt_data, trans, meta, frames = processing_data(self._validation_output_list)
        del self._validation_output_list
        if self.save_tensors:
            tensors = {'prediction': out, 'gt_data': gt_data,
                       'trans': trans, 'metadata': meta, 'frames': frames}
            self._save_tensors(tensors, split_name=self.split, aggr_strategy=self.aggregation_strategy, n_gen=self.n_generated_samples)
        auc_score = self.post_processing(out, gt_data, trans, meta, frames)
        self.log('AUC', auc_score, sync_dist=True)
        return auc_score


    def configure_optimizers(self) -> Dict:
        """
        Configure the optimizers and the learning rate schedulers.

        Returns:
            Dict: dictionary containing the optimizers, the learning rate schedulers and the metric to monitor
        """

        optimizer = Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99, last_epoch=-1)

        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'AUC'}


    def post_processing(self, out: np.ndarray, gt_data: np.ndarray, trans: np.ndarray, meta: np.ndarray, frames: np.ndarray) -> float:
        """
        Post processing of the model.

        Args:
            out (np.ndarray): output of the model
            gt_data (np.ndarray): ground truth data
            trans (np.ndarray): transformation index
            meta (np.ndarray): metadata
            frames (np.ndarray): frame indexes of the data

        Returns:
            float: auc score
        """

        all_gts = [file_name for file_name in os.listdir(self.gt_path) if file_name.endswith('.npy')]
        all_gts = sorted(all_gts)
        scene_clips = [(int(fn.split('_')[0]), int(fn.split('_')[1].split('.')[0])) for fn in all_gts]
        hr_ubnormal_masked_clips = get_hr_ubnormal_mask(self.split) if (self.use_hr and (self.dataset_name == 'UBnormal')) else {}
        hr_avenue_masked_clips = get_avenue_mask() if self.dataset_name == 'HR-Avenue' else {}

        num_transform = self.num_transforms
        model_scores_transf = {}
        dataset_gt_transf = {}

        for transformation in tqdm(range(num_transform)):
            # iterating over each transformation T

            dataset_gt = []
            model_scores = []
            cond_transform = (trans == transformation)

            out_transform, gt_data_transform, meta_transform, frames_transform = filter_vectors_by_cond([out, gt_data, meta, frames], cond_transform)

            for idx in range(len(all_gts)):
                # iterating over each clip C with transformation T

                scene_idx, clip_idx = scene_clips[idx]

                gt = np.load(os.path.join(self.gt_path, all_gts[idx]))
                n_frames = gt.shape[0]

                cond_scene_clip = (meta_transform[:, 0] == scene_idx) & (meta_transform[:, 1] == clip_idx)
                out_scene_clip, gt_scene_clip, meta_scene_clip, frames_scene_clip = filter_vectors_by_cond([out_transform, gt_data_transform,
                                                                                                            meta_transform, frames_transform],
                                                                                                           cond_scene_clip)

                figs_ids = sorted(list(set(meta_scene_clip[:, 2])))
                error_per_person = []
                error_per_person_max_loss = []

                for fig in figs_ids:
                    # iterating over each actor A in each clip C with transformation T

                    cond_fig = (meta_scene_clip[:, 2] == fig)
                    out_fig, _, frames_fig = filter_vectors_by_cond([out_scene_clip, gt_scene_clip, frames_scene_clip], cond_fig)
                    loss_matrix = compute_var_matrix(out_fig, frames_fig, n_frames)
                    fig_reconstruction_loss = np.nanmax(loss_matrix, axis=0)
                    if self.anomaly_score_pad_size != -1:
                        fig_reconstruction_loss = pad_scores(fig_reconstruction_loss, gt, self.anomaly_score_pad_size)

                    error_per_person.append(fig_reconstruction_loss)
                    error_per_person_max_loss.append(max(fig_reconstruction_loss))

                clip_score = np.stack(error_per_person, axis=0)
                clip_score_log = np.log1p(clip_score)
                clip_score = np.mean(clip_score, axis=0) + (np.amax(clip_score_log, axis=0)-np.amin(clip_score_log, axis=0))

                # removing the non-HR frames for UBnormal dataset
                if (scene_idx, clip_idx) in hr_ubnormal_masked_clips:
                    clip_score = clip_score[hr_ubnormal_masked_clips[(scene_idx, clip_idx)]]
                    gt = gt[hr_ubnormal_masked_clips[(scene_idx, clip_idx)]]

                # removing the non-HR frames for Avenue dataset
                if clip_idx in hr_avenue_masked_clips:
                    clip_score = clip_score[np.array(hr_avenue_masked_clips[clip_idx])==1]
                    gt = gt[np.array(hr_avenue_masked_clips[clip_idx])==1]

                # Abnormal score per frame
                clip_score = score_process(clip_score, self.anomaly_score_frames_shift, self.anomaly_score_filter_kernel_size)
                model_scores.append(clip_score)

                dataset_gt.append(gt)

            model_scores = np.concatenate(model_scores, axis=0)

            dataset_gt = np.concatenate(dataset_gt, axis=0)

            model_scores_transf[transformation] = model_scores
            dataset_gt_transf[transformation] = dataset_gt

        # aggregating the anomaly scores for all transformations
        pds = np.mean(np.stack(list(model_scores_transf.values()), 0), 0)
        gt = dataset_gt_transf[0]

        # computing the AUC
        auc = roc_auc_score(gt, pds)

        return auc


    def test_on_saved_tensors(self, split_name: str) -> float:
        """
        Skip the prediction step and test the model on the saved tensors.

        Args:
            split_name (str): split name (val, test)

        Returns:
            float: auc score
        """

        tensors = self._load_tensors(split_name, self.aggregation_strategy, self.n_generated_samples)
        auc_score = self.post_processing(tensors['prediction'], tensors['gt_data'], tensors['trans'],
                                         tensors['metadata'], tensors['frames'])
        print(f'AUC score: {auc_score:.6f}')
        return auc_score


    ## Helper functions

    def _aggregation_strategy(self, generated_xs: List[torch.Tensor], input_sequence: torch.Tensor, aggr_strategy: str) -> Tuple[torch.Tensor]:
        """
        Aggregates the generated samples and returns the selected one and its reconstruction error.
        Strategies:
            - all: returns all the generated samples
            - random: returns a random sample
            - best: returns the sample with the lowest reconstruction loss
            - worst: returns the sample with the highest reconstruction loss
            - mean: returns the mean of the losses of the generated samples
            - median: returns the median of the losses of the generated samples
            - mean_pose: returns the mean of the generated samples
            - median_pose: returns the median of the generated samples

        Args:
            generated_xs (List[torch.Tensor]): list of generated samples
            input_sequence (torch.Tensor): ground truth sequence
            aggr_strategy (str): aggregation strategy

        Raises:
            ValueError: if the aggregation strategy is not valid

        Returns:
            Tuple[torch.Tensor]: selected sample and its reconstruction error
        """

        aggr_strategy = self.aggregation_strategy if aggr_strategy is None else aggr_strategy
        if aggr_strategy == 'random':
            return generated_xs[np.random.randint(len(generated_xs))], None # Added None as it was missing

        B, repr_shape = input_sequence.shape[0], input_sequence.shape[1:]
        compute_loss = lambda x: torch.mean(self.loss_fn(x, input_sequence).reshape(-1, prod(repr_shape)), dim=-1)
        losses = [compute_loss(x) for x in generated_xs]

        if aggr_strategy == 'all':
            dims_idxs = list(range(2, len(repr_shape)+2))
            dims_idxs = [1, 0] + dims_idxs
            selected_x = torch.stack(generated_xs).permute(*dims_idxs)
            loss_of_selected_x = torch.stack(losses).permute(1, 0)
        elif aggr_strategy == 'mean':
            selected_x = None
            loss_of_selected_x = torch.mean(torch.stack(losses), dim=0)
        elif aggr_strategy == 'mean_pose':
            selected_x = torch.mean(torch.stack(generated_xs), dim=0)
            loss_of_selected_x = compute_loss(selected_x)
        elif aggr_strategy == 'median':
            loss_of_selected_x, _ = torch.median(torch.stack(losses), dim=0)
            selected_x = None
        elif aggr_strategy == 'median_pose':
            selected_x, _ = torch.median(torch.stack(generated_xs), dim=0)
            loss_of_selected_x = compute_loss(selected_x)
        elif aggr_strategy == 'best' or aggr_strategy == 'worst':
            strategy = (lambda x, y: x < y) if aggr_strategy == 'best' else (lambda x, y: x > y)
            loss_of_selected_x = torch.full((B,), fill_value=(1e10 if aggr_strategy == 'best' else -1.), device=self.device)
            selected_x = torch.zeros((B, *repr_shape)).to(self.device)

            for i in range(len(generated_xs)):
                mask = strategy(losses[i], loss_of_selected_x)
                loss_of_selected_x[mask] = losses[i][mask]
                selected_x[mask] = generated_xs[i][mask]
        elif 'quantile' in aggr_strategy:
            q = float(aggr_strategy.split(':')[-1])
            loss_of_selected_x = torch.quantile(torch.stack(losses), q, dim=0)
            selected_x = None
        else:
            raise ValueError(f'Unknown aggregation strategy {aggr_strategy}')

        # Ensuring selected_x and loss_of_selected_x are always returned
        if selected_x is None and loss_of_selected_x is None:
             # Default to mean loss if strategy doesn't return both
             loss_of_selected_x = torch.mean(torch.stack(losses), dim=0)


        return selected_x, loss_of_selected_x


    def _infer_number_of_joint(self, args: argparse.Namespace) -> int:
        """
        Infer the number of joints based on the dataset parameters.

        Args:
            args (argparse.Namespace): arguments containing the hyperparameters of the model

        Returns:
            int: number of joints
        """

        if args.headless:
            joints_to_consider = 14
        elif args.kp18_format:
            joints_to_consider = 18
        else:
            joints_to_consider = 17
        return joints_to_consider


    def _load_tensors(self, split_name: str, aggr_strategy: str, n_gen: int) -> Dict[str, torch.Tensor]:
        """
        Loads the tensors from the experiment directory.

        Args:
            split_name (str): name of the split (train, val, test)
            aggr_strategy (str): aggregation strategy
            n_gen (int): number of generated samples

        Returns:
            Dict[str, torch.Tensor]: dictionary containing the tensors. The keys are inferred from the file names.
        """

        name = 'saved_tensors_{}_{}_{}'.format(split_name, aggr_strategy, n_gen)
        path = os.path.join(self.ckpt_dir, name)
        if not os.path.exists(path):
            os.mkdir(path)
        tensor_files = os.listdir(path)
        tensors = {}
        for t_file in tensor_files:
            t_name = t_file.split('.')[0]
            tensors[t_name] = torch.load(os.path.join(path, t_file))
        return tensors


    def _pack_out_data(self, selected_x: torch.Tensor, loss_of_selected_x: torch.Tensor, additional_out: List[torch.Tensor], return_: str) -> List[torch.Tensor]:
        """
        Packs the output data according to the return_ argument.

        Args:
            selected_x (torch.Tensor): generated samples selected among the others according to the aggregation strategy
            loss_of_selected_x (torch.Tensor): loss of the selected samples
            additional_out (List[torch.Tensor]): additional output data (ground truth, meta-data, etc.)
            return_ (str): return strategy. Can be 'pose', 'loss', 'all'

        Raises:
            ValueError: if return_ is None and self.model_return_value is None

        Returns:
            List[torch.Tensor]: output data
        """

        if return_ is None:
            if self.model_return_value is None:
                raise ValueError('Either return_ or self.model_return_value must be set')
            else:
                return_ = self.model_return_value

        if return_ == 'poses':
            out = [selected_x]
        elif return_ == 'loss':
            out = [loss_of_selected_x]
        elif return_ == 'all':
            # Check if both are available before adding to the list
            out = []
            if loss_of_selected_x is not None:
                out.append(loss_of_selected_x)
            if selected_x is not None:
                 out.append(selected_x)

        return out + additional_out


    def _save_tensors(self, tensors: Dict[str, torch.Tensor], split_name: str, aggr_strategy: str, n_gen: int) -> None:
        """
        Saves the tensors in the experiment directory.

        Args:
            tensors (Dict[str, torch.Tensor]): tensors to save
            split_name (str): name of the split (val, test)
            aggr_strategy (str): aggregation strategy
            n_gen (int): number of generated samples
        """

        name = 'saved_tensors_{}_{}_{}'.format(split_name, aggr_strategy, n_gen)
        path = os.path.join(self.ckpt_dir, name)
        if not os.path.exists(path):
            os.mkdir(path)
        for t_name, tensor in tensors.items():
            torch.save(tensor, os.path.join(path, t_name + '.pt'))


    def _set_diffusion_variables(self) -> None:
        """
        Sets the diffusion variables.
        """

        self.noise_scheduler = Diffusion(noise_steps=self.noise_steps, n_joints=self.n_joints,
                                         device=self.device, time=self.n_frames)
        self._beta_ = self.noise_scheduler.schedule_noise()
        self._alpha_ = (1. - self._beta_)
        self._alpha_hat_ = torch.cumprod(self._alpha_, dim=0)

    def _unpack_data(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        """
        Unpacks the data.

        Args:
            x (torch.Tensor): list containing the input data, the transformation index, the metadata and the actual frames.

        Returns:
            Tuple[torch.Tensor, List[torch.Tensor]]: input data, list containing the transformation index, the metadata and the actual frames.
        """
        tensor_data = x[0].to(self.device)
        transformation_idx = x[1]
        metadata = x[2]
        actual_frames = x[3]
        meta_out = [transformation_idx, metadata, actual_frames]
        return tensor_data, meta_out


    @property
    def _beta(self) -> torch.Tensor:
        return self._beta_.to(self.device)


    @property
    def _alpha(self) -> torch.Tensor:
        return self._alpha_.to(self.device)


    @property
    def _alpha_hat(self) -> torch.Tensor:
        return self._alpha_hat_.to(self.device)

Overwriting /kaggle/working/DCMD-main/models/dcmd.py


In [13]:
%%writefile /kaggle/working/DCMD-main/eval_DCMD.py
import argparse
import os

import pytorch_lightning as pl
import yaml
from models.dcmd import DCMD
from utils.argparser import init_args
from utils.dataset import get_dataset_and_loader



if __name__== '__main__':
    
    # Parse command line arguments and load config file
    parser = argparse.ArgumentParser(description='DCMD')
    parser.add_argument('-c', '--config', type=str, required=True)
    args = parser.parse_args()
    args = yaml.load(open(args.config), Loader=yaml.FullLoader)
    args = argparse.Namespace(**args)
    args = init_args(args)

    # Initialize the model
    model = DCMD(args)
    
    if args.load_tensors:
        # Load tensors and test
        model.test_on_saved_tensors(split_name=args.split)
    else:
        # Load test data
        print('Loading data and creating loaders.....')
        ckpt_path = '/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/last.ckpt'
        dataset, loader, _, _ = get_dataset_and_loader(args, split=args.split)
        
        # Initialize trainer and test
        trainer = pl.Trainer(accelerator=args.accelerator, devices=args.devices[:1],
                             default_root_dir=args.ckpt_dir, max_epochs=1, logger=False)
        out = trainer.test(model, dataloaders=loader, ckpt_path=ckpt_path)

Overwriting /kaggle/working/DCMD-main/eval_DCMD.py


In [14]:
%%writefile /kaggle/working/DCMD-main/utils/get_robust_data.py
import os
import numpy as np
import pickle

from copy import deepcopy

import torch

from utils.data import load_trajectories, extract_global_features
from utils.data import change_coordinate_system, scale_trajectories, aggregate_autoencoder_data
from utils.data import input_trajectories_missing_steps
from utils.preprocessing import remove_short_trajectories, aggregate_rnn_autoencoder_data


def save_scaler(scaler, path):
    with open(path, 'wb') as scaler_file:
        pickle.dump(scaler, scaler_file)
    
        
def load_scaler(path):
    with open(path, 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    return scaler



# Load trajectory data and convert it into a format suitable for RNN autoencoders for training and testing of joint models
def data_of_combined_model(**args):
    # General
    exp_dir = args.get('exp_dir', '')
    split = args.get('split', 'train')
    normalize_pose = args.get('normalize_pose', False)
    trajectories_path = args.get('trajectories_path', '')
    include_global = args.get('include_global', True)
    debug = args.get('debug', False)
    if 'train' in split:
      # đổi thành path khác trong folder data nếu train dataset khác
        subfolder = '/kaggle/input/avenue/Avenue/training'
    elif 'test' in split:
        subfolder = '/kaggle/input/avenue/Avenue/testing'
    else:
        subfolder = 'validating'
    trajectories_path = os.path.join(trajectories_path, f'{subfolder}/trajectories')
    video_resolution = args.get('vid_res', (1080,720))
    video_resolution = np.array(video_resolution, dtype=np.float32)
    # Architecture
    reconstruct_original_data = args.get('reconstruct_original_data', False) 
    input_length = args.get('seg_len', 12)
    seg_stride = args.get('seg_stride', 1) - 1 
    pred_length = 0 
    # Training
    input_missing_steps = False # args.input_missing_steps
    
    if normalize_pose == True:
        global_normalisation_strategy = args.get('normalization_strategy', 'robust')
        local_normalisation_strategy = args.get('normalization_strategy', 'robust')
        out_normalisation_strategy = args.get('normalization_strategy', 'robust')


    trajectories = load_trajectories(trajectories_path, debug=debug, split=split)
    print('\nLoaded %d trajectories.' % len(trajectories))

    trajectories = remove_short_trajectories(trajectories, input_length=input_length,
                                             input_gap=seg_stride, pred_length=pred_length)
    print('\nRemoved short trajectories. Number of trajectories left: %d.' % len(trajectories))

    # trajectories, trajectories_val = split_into_train_and_test(trajectories, train_ratio=0.8, seed=42)

    if input_missing_steps:
        trajectories = input_trajectories_missing_steps(trajectories)
        print('\nInputted missing steps of trajectories.')

    # Global
    if include_global:
        global_trajectories = extract_global_features(deepcopy(trajectories), video_resolution=video_resolution)

        global_trajectories = change_coordinate_system(global_trajectories, video_resolution=video_resolution,
                                                        coordinate_system='global', invert=False)

        print('\nChanged global trajectories\'s coordinate system to global.')
        
        X_global, y_global, X_global_meta, y_global_meta = aggregate_rnn_autoencoder_data(global_trajectories, 
                                                                                        input_length=input_length,
                                                                                        input_gap=seg_stride, pred_length=pred_length, 
                                                                                        return_ids=True)
        
        if normalize_pose == True:
            # nếu test avenue thì dùng dòng này
            #/content/checkpoints/HR-Avenue/train_experiment/local_robust.pickle
            # default: scaler_path = os.path.join(exp_dir, f'global_{global_normalisation_strategy}.pickle')
            scaler_path = '/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/local_robust.pickle'
            if split == 'train':
                _, global_scaler = scale_trajectories(aggregate_autoencoder_data(global_trajectories),
                                                    strategy=global_normalisation_strategy)
                save_scaler(global_scaler, scaler_path)
            else:
                global_scaler = load_scaler(scaler_path)

            X_global, _ = scale_trajectories(X_global, scaler=global_scaler, strategy=global_normalisation_strategy)
            
            if y_global is not None:
                y_global, _ = scale_trajectories(y_global, scaler=global_scaler,
                                                strategy=global_normalisation_strategy)
                
            print('\nNormalised global trajectories using the %s normalisation strategy.' % global_normalisation_strategy)
    
    else:
        X_global, X_global_meta = None, None
    
    # Local
    local_trajectories = deepcopy(trajectories) if reconstruct_original_data else trajectories

    local_trajectories = change_coordinate_system(local_trajectories, video_resolution=video_resolution,
                                                  coordinate_system='bounding_box_centre', invert=False)

    print('\nChanged local trajectories\'s coordinate system to bounding_box_centre.')

    X_local, y_local, X_local_meta, y_local_meta = aggregate_rnn_autoencoder_data(local_trajectories, input_length=input_length, 
                                                                                  input_gap=seg_stride, pred_length=pred_length,
                                                                                  return_ids=True)
    
    if normalize_pose == True:
        #scaler_path = '/content/drive/MyDrive/DCMD-main-main/checkpoints/Avenue/test_experiment/local_robust.pickle'
        scaler_path = os.path.join(exp_dir, f'local_{local_normalisation_strategy}.pickle')

        if split == 'train':
            _, local_scaler = scale_trajectories(aggregate_autoencoder_data(local_trajectories),
                                                strategy=local_normalisation_strategy)
            save_scaler(local_scaler, scaler_path)
        else:
            local_scaler = load_scaler("/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/local_robust.pickle")

        X_local, _ = scale_trajectories(X_local, scaler=local_scaler, strategy=local_normalisation_strategy)

        if y_local is not None:
            y_local, _ = scale_trajectories(y_local, scaler=local_scaler, strategy=local_normalisation_strategy)
        
        print('\nNormalised local trajectories using the %s normalisation strategy.' % local_normalisation_strategy)

    # (Optional) Reconstruct the original data
    if reconstruct_original_data:
        print('\nReconstruction/Prediction target is the original data.')
        out_trajectories = trajectories
        
        out_trajectories = change_coordinate_system(out_trajectories, video_resolution=video_resolution,
                                                    coordinate_system='global', invert=False)
    
        print('\nChanged target trajectories\'s coordinate system to global.')
        
        scaler_path = os.path.join(exp_dir, f'out_{out_normalisation_strategy}.pickle')
    
        if split == 'train':
            _, out_scaler = scale_trajectories(aggregate_autoencoder_data(out_trajectories),
                                               strategy=out_normalisation_strategy)
            save_scaler(out_scaler, scaler_path)
        else:
            out_scaler = load_scaler(scaler_path)
        
        ######## X_out_{}, y_out_{} numpy arrays

        X_out, y_out, X_out_meta, y_out_meta = aggregate_rnn_autoencoder_data(out_trajectories, input_length=input_length, 
                                                                              input_gap=seg_stride, pred_length=pred_length,
                                                                              return_ids=True)

        X_out, _ = scale_trajectories(X_out, scaler=out_scaler, strategy=out_normalisation_strategy)
        
        if y_out is not None:
            y_out, _ = scale_trajectories(y_out, scaler=out_scaler, strategy=out_normalisation_strategy)
            
        print('\nNormalised target trajectories using the %s normalisation strategy.' % out_normalisation_strategy)
        
            
    if pred_length > 0:
        
        if reconstruct_original_data:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (X_out, X_out_meta), \
                   (y_global, y_global_meta), \
                   (y_local, y_local_meta), \
                   (y_out, y_out_meta) 
        else:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (y_global, y_global_meta), \
                   (y_local, y_local_meta)
    else:
        if reconstruct_original_data:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (X_out, X_out_meta)
        else:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta)

Overwriting /kaggle/working/DCMD-main/utils/get_robust_data.py


# **Train** 

In [ ]:
!python /kaggle/working/DCMD-main/train_DCMD.py --config /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml

If download checkpoint needed, run this code and download checkpoints.zip file in Output

In [ ]:
# 1. Zip toàn bộ folder checkpoints
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/DCMD-main/checkpoints

# 2. (tuỳ chọn) kiểm tra xem zip đã tạo xong chưa
!ls -lh /kaggle/working/checkpoints.zip


# **Testing**

In [18]:
import os
os.makedirs('/kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment', exist_ok=True)

In [19]:
!python /kaggle/working/DCMD-main/eval_DCMD.py --config /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml

Loading data and creating loaders.....

Loaded 2594 trajectories.

Removed short trajectories. Number of trajectories left: 1661.

Changed local trajectories's coordinate system to bounding_box_centre.
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.3.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(

Normalised local trajectories using the robust normalisation strategy.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py:282: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': 'loss', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epo

In [22]:
# 1. Zip toàn bộ folder checkpoints
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/DCMD-main/checkpoints



updating: kaggle/working/DCMD-main/checkpoints/ (stored 0%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/ (stored 0%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/ (stored 0%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/ (stored 0%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/prediction.pt (deflated 25%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/trans.pt (deflated 100%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/metadata.pt (deflated 90%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/gt_data.pt (deflated 89%)
updating: kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment/saved_tensors_test_best_50/frames.pt (deflated 96%)
